In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/security/new_merge.csv")
df.sample(5)

<ipython-input-3-71bacd3ba159>:1: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/security/new_merge.csv")


,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
908057,363989,172.16.0.5-192.168.50.1-59414-80-6,192.168.50.1,80,172.16.0.5,59414,6,2018-12-01 13:19:48.652617,93,1,...,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,WebDDoS
924505,334964,172.16.0.5-192.168.50.1-60365-60365-6,172.16.0.5,60365,192.168.50.1,60365,6,2018-12-01 13:29:44.659404,45146085,8,...,76.367532,109.0,1.0,15048658.0,2.110127e+06,17214167.0,12998657.0,0,1,UDP-lag
673028,18196,172.16.0.5-192.168.50.1-37338-27786-17,172.16.0.5,37338,192.168.50.1,27786,17,2018-12-01 12:54:58.888945,1,2,...,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,1,DrDoS_UDP
903634,103279,172.16.0.5-192.168.50.1-24364-24364-17,172.16.0.5,24364,192.168.50.1,24364,17,2018-12-01 13:11:14.396717,10,2,...,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,1,UDP-lag
245836,4693,172.16.0.5-192.168.50.1-634-64872-17,172.16.0.5,634,192.168.50.1,64872,17,2018-12-01 10:36:32.810133,8865,36,...,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,1,DrDoS_NTP


In [ ]:
df = df.drop(['Unnamed: 0'], axis=1)
df.sample(5)

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
668123,172.16.0.5-192.168.50.1-38376-61439-17,172.16.0.5,38376,192.168.50.1,61439,17,2018-12-01 12:54:56.754613,3,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_UDP
521300,172.16.0.5-192.168.50.1-41914-11233-17,172.16.0.5,41914,192.168.50.1,11233,17,2018-12-01 12:27:13.950766,106891,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_SSDP
62068,172.16.0.5-192.168.50.1-900-37763-17,172.16.0.5,900,192.168.50.1,37763,17,2018-12-01 11:22:56.434376,1,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_LDAP
298778,172.16.0.5-192.168.50.1-41771-36791-17,172.16.0.5,41771,192.168.50.1,36791,17,2018-12-01 11:47:09.785401,48,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_NetBIOS
150370,192.168.50.8-54.222.149.110-58358-443-6,54.222.149.110,443,192.168.50.8,58358,6,2018-12-01 09:19:28.590644,1063621,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,BENIGN


In [ ]:
num_of_label = df.groupby(' Label').size()
num_of_label

,0
Label,
BENIGN,24701
DrDoS_LDAP,148198
DrDoS_NTP,123591
DrDoS_NetBIOS,145922
DrDoS_SSDP,146107
DrDoS_UDP,131730
Syn,138539
UDP-lag,132720
WebDDoS,33453


In [ ]:
ddos_type_map = {}
for i in range(len(num_of_label)):
    ddos_type_map[num_of_label.index[i]] = i
ddos_type_map

{'BENIGN': 0,
 'DrDoS_LDAP': 1,
 'DrDoS_NTP': 2,
 'DrDoS_NetBIOS': 3,
 'DrDoS_SSDP': 4,
 'DrDoS_UDP': 5,
 'Syn': 6,
 'UDP-lag': 7,
 'WebDDoS': 8}

In [ ]:
df[' Label'] = df[' Label'].map(ddos_type_map)
df.sample(5)

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
616426,172.16.0.5-192.168.50.1-44857-4705-17,172.16.0.5,44857,192.168.50.1,4705,17,2018-12-01 12:37:02.745912,1,2,0,...,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,1,5
966139,172.16.0.5-192.168.50.1-36189-62367-6,172.16.0.5,36189,192.168.50.1,62367,6,2018-12-01 13:29:49.653659,103,2,2,...,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,1,7
914848,172.16.0.5-192.168.50.1-29864-29864-6,172.16.0.5,29864,192.168.50.1,29864,6,2018-12-01 13:29:43.484275,43549142,6,0,...,35.355339,51.0,1.0,21774543.5,4.593373e+06,25022549.0,18526538.0,0,1,7
983451,172.16.0.5-192.168.50.1-47000-39733-6,172.16.0.5,47000,192.168.50.1,39733,6,2018-12-01 13:29:51.774321,1,2,0,...,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,1,7
383,172.217.11.42-192.168.50.1-443-57524-6,192.168.50.1,57524,172.217.11.42,443,6,2018-12-01 11:25:27.586082,32584,4,2,...,0.000000,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0,0,0


In [ ]:
numeric_col = list(df.select_dtypes("number").columns)

In [ ]:
numeric_col

[' Source Port',
 ' Destination Port',
 ' Protocol',
 ' Flow Duration',
 ' Total Fwd Packets',
 ' Total Backward Packets',
 'Total Length of Fwd Packets',
 ' Total Length of Bwd Packets',
 ' Fwd Packet Length Max',
 ' Fwd Packet Length Min',
 ' Fwd Packet Length Mean',
 ' Fwd Packet Length Std',
 'Bwd Packet Length Max',
 ' Bwd Packet Length Min',
 ' Bwd Packet Length Mean',
 ' Bwd Packet Length Std',
 'Flow Bytes/s',
 ' Flow Packets/s',
 ' Flow IAT Mean',
 ' Flow IAT Std',
 ' Flow IAT Max',
 ' Flow IAT Min',
 'Fwd IAT Total',
 ' Fwd IAT Mean',
 ' Fwd IAT Std',
 ' Fwd IAT Max',
 ' Fwd IAT Min',
 'Bwd IAT Total',
 ' Bwd IAT Mean',
 ' Bwd IAT Std',
 ' Bwd IAT Max',
 ' Bwd IAT Min',
 'Fwd PSH Flags',
 ' Bwd PSH Flags',
 ' Fwd URG Flags',
 ' Bwd URG Flags',
 ' Fwd Header Length',
 ' Bwd Header Length',
 'Fwd Packets/s',
 ' Bwd Packets/s',
 ' Min Packet Length',
 ' Max Packet Length',
 ' Packet Length Mean',
 ' Packet Length Std',
 ' Packet Length Variance',
 'FIN Flag Count',
 ' SYN Flag C

In [ ]:
new_data = df[numeric_col]
new_data.describe()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Inbound,Label
count,1.024961e+06,1.024961e+06,1.024961e+06,1.024961e+06,1.024961e+06,1.024961e+06,1.024961e+06,1.024961e+06,1.024961e+06,1.024961e+06,...,1.024961e+06,1.024961e+06,1.024961e+06,1.024961e+06,1.024961e+06,1.024961e+06,1.024961e+06,1.024961e+06,1.024961e+06,1.024961e+06
mean,2.524254e+04,3.177504e+04,1.404100e+01,6.768858e+06,1.216897e+01,4.301422e-01,4.764924e+03,2.102648e+02,4.938198e+02,4.704957e+02,...,4.685199e+03,4.580040e+03,1.147050e+04,2.208643e+03,1.732375e+06,4.724207e+05,2.336782e+06,1.295394e+06,9.657782e-01,4.004174e+00
std,2.373271e+04,1.967316e+04,4.885000e+00,2.333196e+07,1.990309e+02,6.937314e+00,1.394516e+04,2.296371e+04,5.065392e+02,5.118934e+02,...,1.396480e+05,1.062776e+05,2.397990e+05,1.160583e+05,5.748698e+06,1.955576e+06,7.787560e+06,4.610844e+06,1.817985e-01,2.160638e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.670000e+02,1.460800e+04,6.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00
50%,2.419400e+04,3.195300e+04,1.700000e+01,2.000000e+00,2.000000e+00,0.000000e+00,8.780000e+02,0.000000e+00,3.890000e+02,3.300000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,4.000000e+00
75%,4.784100e+04,4.901200e+04,1.700000e+01,3.613200e+04,6.000000e+00,0.000000e+00,2.944000e+03,0.000000e+00,4.440000e+02,4.400000e+02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,6.000000e+00
max,6.553200e+04,6.553500e+04,1.700000e+01,1.200000e+08,8.617600e+04,1.862000e+03,1.760000e+05,8.829488e+06,3.569000e+03,2.020000e+03,...,6.151289e+07,4.868047e+07,7.286843e+07,6.151289e+07,1.166865e+08,5.727195e+07,1.166865e+08,1.166865e+08,1.000000e+00,8.000000e+00


In [ ]:
# def rm_outliers(col_name):
#     q1 = df[col_name].quantile(0.25)
#     q3 = df[col_name].quantile(0.75)
#     IQR = q3 - q1
#     lower_bound = q1 - 1.5*IQR
#     upper_bound = q3 + 1.5*IQR
#     df_result = df[(df[col_name] >= lower_bound) & (df[col_name] <= upper_bound)]
#     return df_result

In [ ]:
# for col in numeric_col:
#     new_data = rm_outliers(col)

In [ ]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1024961 entries, 0 to 1024960
Data columns (total 82 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0    Source Port                  1024961 non-null  int64  
 1    Destination Port             1024961 non-null  int64  
 2    Protocol                     1024961 non-null  int64  
 3    Flow Duration                1024961 non-null  int64  
 4    Total Fwd Packets            1024961 non-null  int64  
 5    Total Backward Packets       1024961 non-null  int64  
 6   Total Length of Fwd Packets   1024961 non-null  float64
 7    Total Length of Bwd Packets  1024961 non-null  float64
 8    Fwd Packet Length Max        1024961 non-null  float64
 9    Fwd Packet Length Min        1024961 non-null  float64
 10   Fwd Packet Length Mean       1024961 non-null  float64
 11   Fwd Packet Length Std        1024961 non-null  float64
 12  Bwd Packet Length Max       

- Using **corr()** to retrieve correlations between columns
- Input Features will be decided based on the correlation between the column with the target feature (**Label**)

In [ ]:
new_data.corr()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Inbound,Label
Source Port,1.000000,-0.028910,-0.239186,0.144522,-0.040023,0.046887,-0.285367,0.010063,-0.439278,-0.449309,...,0.018940,0.019827,0.023965,0.013170,0.151233,0.116177,0.148908,0.143037,-0.059474,0.519664
Destination Port,-0.028910,1.000000,0.073370,-0.006094,-0.003372,-0.060592,0.014544,-0.014684,0.032934,0.054648,...,-0.024738,-0.010050,-0.019789,-0.023575,-0.011019,0.008984,-0.004920,-0.017493,0.127736,0.005360
Protocol,-0.239186,0.073370,1.000000,-0.475506,0.011147,-0.097161,0.202132,-0.014871,0.554659,0.556587,...,-0.054049,-0.072204,-0.078964,-0.029298,-0.495135,-0.397639,-0.493220,-0.461302,0.175815,-0.602009
Flow Duration,0.144522,-0.006094,-0.475506,1.000000,0.004637,0.092162,-0.095693,0.011191,-0.267386,-0.265539,...,0.079252,0.150166,0.154096,0.024880,0.833760,0.787000,0.896743,0.714675,0.004471,0.273469
Total Fwd Packets,-0.040023,-0.003372,0.011147,0.004637,1.000000,0.035809,0.156473,0.009080,-0.010623,-0.012169,...,0.016696,0.010522,0.015366,0.013544,-0.001500,-0.001012,-0.001034,-0.001755,0.006876,-0.036304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Idle Std,0.116177,0.008984,-0.397639,0.787000,-0.001012,0.040947,-0.081576,-0.000573,-0.231808,-0.221633,...,0.045112,0.102725,0.094197,0.004495,0.728346,1.000000,0.857589,0.542277,0.033066,0.250933
Idle Max,0.148908,-0.004920,-0.493220,0.896743,-0.001034,0.085728,-0.099297,0.009100,-0.277807,-0.275243,...,0.080231,0.126270,0.127142,0.033683,0.968525,0.857589,1.000000,0.882358,0.000634,0.301914
Idle Min,0.143037,-0.017493,-0.461302,0.714675,-0.001755,0.104634,-0.091289,0.016355,-0.251855,-0.257626,...,0.094386,0.112403,0.119380,0.054165,0.966209,0.542277,0.882358,1.000000,-0.032080,0.283970
Inbound,-0.059474,0.127736,0.175815,0.004471,0.006876,-0.134022,0.057538,-0.040768,0.137289,0.159473,...,-0.057357,-0.020595,-0.044406,-0.056235,-0.015042,0.033066,0.000634,-0.032080,1.000000,0.091408


In [ ]:
corr_matrix = new_data.corr()

- Retrive all columns which have correlations between 0.3 and 0.9
- As these columns are believed to have impact to the target feature - **Label**

In [ ]:
corr_values = corr_matrix[" Label"]  # Replace "Label" with your actual label column name

# Filter correlation values between 0.3 and 0.9 or between -0.3 and -0.9
filtered_corr = corr_values[((corr_values > 0.3) & (corr_values < 0.9)) | ((corr_values < -0.3) & (corr_values > -0.9))]

# Display columns with correlation in the specified range
print(filtered_corr)

 Source Port               0.519664
 Protocol                 -0.602009
 Fwd Packet Length Max    -0.602659
 Fwd Packet Length Min    -0.594040
 Fwd Packet Length Mean   -0.606234
Flow Bytes/s              -0.515040
 Flow IAT Mean             0.340307
 Flow IAT Std              0.339467
 Flow IAT Max              0.314120
 Fwd IAT Mean              0.341705
 Fwd IAT Std               0.334408
 Fwd IAT Max               0.310049
 Min Packet Length        -0.594872
 Max Packet Length        -0.562291
 Packet Length Mean       -0.607841
 ACK Flag Count            0.588518
 Average Packet Size      -0.581971
 Avg Fwd Segment Size     -0.606234
Idle Mean                  0.304900
 Idle Max                  0.301914
Name:  Label, dtype: float64


In [ ]:
filtered_corr.index

Index([' Source Port', ' Protocol', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean', 'Flow Bytes/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Fwd IAT Mean',
       ' Fwd IAT Std', ' Fwd IAT Max', ' Min Packet Length',
       ' Max Packet Length', ' Packet Length Mean', ' ACK Flag Count',
       ' Average Packet Size', ' Avg Fwd Segment Size', 'Idle Mean',
       ' Idle Max'],
      dtype='object')

In [ ]:
# Split data into features and target
X = new_data[filtered_corr.index]
y = new_data[" Label"]

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_test = X_test.replace([np.inf, -np.inf], np.nan)

X_train = X_train.dropna()
X_test = X_test.dropna()

y_train = y_train[X_train.index]
y_test = y_test[X_test.index]

# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

# Make predictions
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.7907910378594024

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      4972
           1       0.92      0.98      0.95     28896
           2       0.99      0.99      0.99     24522
           3       0.98      0.98      0.98     28136
           4       0.57      0.53      0.55     28503
           5       0.54      0.59      0.56     26329
           6       0.81      0.83      0.82     24959
           7       0.62      0.54      0.58     25586
           8       0.98      1.00      0.99      6621

    accuracy                           0.79    198524
   macro avg       0.82      0.83      0.82    198524
weighted avg       0.79      0.79      0.79    198524


Confusion Matrix:
 [[ 4919     0    14     1     1     3     8     2    24]
 [    4 28305     3     6   574     1     1     0     2]
 [   31     1 24336     3     7    19     7    16   102]
 [    1    12    15 27712   200   128     0    61 

In [ ]:
X_test.iloc[0]

,110289
Source Port,668.0
Flow Duration,1.0
Flow IAT Mean,1.0
Flow IAT Std,0.0
Flow IAT Max,1.0
Fwd IAT Total,1.0
Fwd IAT Mean,1.0
Fwd IAT Std,0.0
Fwd IAT Max,1.0
ACK Flag Count,0.0


In [ ]:
test_data = new_data[new_data[" Label"] == 7]
test_data = test_data[filtered_corr.index]
test_data = test_data.sample(5)

In [ ]:
test_data = df.drop([" Label"], axis=1)

In [ ]:
test_data.sample(5)

,Source Port,Protocol,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Flow Bytes/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Min Packet Length,Max Packet Length,Packet Length Mean,ACK Flag Count,Average Packet Size,Avg Fwd Segment Size,Idle Mean,Idle Max
680332,48461,17,389.0,330.0,359.5,1.353896e+04,35404.0,61318.928717,106209.0,35404.0,61318.928717,106209.0,330.0,389.0,353.600000,0,442.0,359.5,0.0,0.0
676358,59067,17,393.0,321.0,348.0,9.678855e+03,43145.6,59097.846025,110043.0,43145.6,59097.846025,110043.0,321.0,393.0,344.142857,0,401.5,348.0,0.0,0.0
630397,38209,17,383.0,383.0,383.0,1.629787e+07,47.0,0.000000,47.0,47.0,0.000000,47.0,383.0,383.0,383.000000,0,574.5,383.0,0.0,0.0
665591,54089,17,383.0,383.0,383.0,7.660000e+08,1.0,0.000000,1.0,1.0,0.000000,1.0,383.0,383.0,383.000000,0,574.5,383.0,0.0,0.0
654824,672,17,1472.0,1472.0,1472.0,9.813333e+08,3.0,0.000000,3.0,3.0,0.000000,3.0,1472.0,1472.0,1472.000000,0,2208.0,1472.0,0.0,0.0


In [ ]:
test_input = {}

for col in df.columns:
    test_input[col] = df.iloc[680332][col]

test_input = pd.DataFrame(test_input, index=[0])
test_input = test_input[filtered_corr.index]

In [ ]:
# Reverse the dictionary for decoding
reverse_map = {v: k for k, v in ddos_type_map.items()}

In [ ]:
prediction = rf_classifier.predict(test_input)
print(f"Predicted: {reverse_map[prediction[0]]}")

Predicted: DrDoS_UDP


In [ ]:
predictions = rf_classifier.predict(test_data)
print("\nPredicted Classes:")
print(predictions)

decoded_labels = [reverse_map[pred] for pred in predictions]

# Display results
print("Type:", decoded_labels)


Predicted Classes:
[7 7 7 7 7]
Type: ['UDP-lag', 'UDP-lag', 'UDP-lag', 'UDP-lag', 'UDP-lag']


In [ ]:
def classify_ddos(input):
    input = pd.DataFrame(input, index=[0])
    input = input[filtered_corr.index]
    predictions = rf_classifier.predict(test_data)
    return reverse_map[predictions[0]]

In [ ]:
test_input = {}

for col in df.columns:
    test_input[col] = df.iloc[680332][col]

classification = classify_ddos(test_input)
print(f"Predicted: {classification}")

In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/security/final_combined_data.csv")

<ipython-input-17-5a1ec8b59381>:1: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv("/content/drive/MyDrive/security/final_combined_data.csv")


In [ ]:
df_test_UDP = df_test[df_test[" Label"] == "DrDoS_UDP"]
df_test_UDP.sample(5)

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
713541,16417,172.16.0.5-192.168.50.1-57166-2531-17,172.16.0.5,57166,192.168.50.1,2531,17,2018-12-01 12:55:14.266270,109091,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_UDP
710229,84542,172.16.0.5-192.168.50.1-54193-41559-17,172.16.0.5,54193,192.168.50.1,41559,17,2018-12-01 12:55:12.863503,988999,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_UDP
602489,61582,172.16.0.5-192.168.50.1-55610-61256-17,172.16.0.5,55610,192.168.50.1,61256,17,2018-12-01 12:37:00.206330,105703,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_UDP
613614,57724,172.16.0.5-192.168.50.1-57450-25713-17,172.16.0.5,57450,192.168.50.1,25713,17,2018-12-01 12:37:02.149037,49,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_UDP
617085,51985,172.16.0.5-192.168.50.1-44054-50855-17,172.16.0.5,44054,192.168.50.1,50855,17,2018-12-01 12:37:02.891750,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,DrDoS_UDP
